# Overview

## Import Libraries

In [248]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn #neural network
import torchvision as tv
import torchvision.transforms as TF
import torch.optim as optim # optimizer
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Set Cuda Device and Import Data

In [557]:
# set test device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [558]:
index_df = pd.read_csv("../../data/image_index.csv", index_col=0)
index_df.head()

,img,data_set,condition
0,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0
1,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0
2,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0
3,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0
4,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0


In [559]:
result_dummy = pd.get_dummies(index_df.condition)

In [560]:
result_dummy.columns = ["negative", "positive"]

In [561]:
index_df = pd.concat([index_df, result_dummy], axis=1)

In [562]:
index_df.head()

,img,data_set,condition,negative,positive
0,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0,1,0
1,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0,1,0
2,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0,1,0
3,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0,1,0
4,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0,1,0


## Make Train Test Split

In [563]:
test_df = index_df[index_df.data_set==0]
train_df = index_df[index_df.data_set==1]

In [564]:
train_data = train_df[["img", "negative", "positive"]].sample(30, random_state = 42)

In [566]:
y_true = torch.tensor(train_data[["negative", "positive"]].values).float()

In [567]:
img_paths = train_data.img.values

In [568]:
y_true.shape

torch.Size([30, 2])

## Preprocess Data for Modeling

In [498]:
## update this to sequential?
def preprocess_image_flat(path_list, img_h=640, img_w=640):
    outlist = []
    for path in path_list:
        resizer = TF.Resize((img_h, img_w)) #define resizer per new_h and new_w
        im = tv.io.read_image(path).type(torch.float) #read image as pytorch float tensor
        im = resizer(im) #resize image
        normalizer = TF.Normalize(im.mean(), im.std()) #initialize normalizer
        im = normalizer(im) # return normalized pytorch float tensor
        im = torch.flatten(im)
        outlist.append(im)
    return torch.stack(outlist)

In [499]:
input_data = preprocess_image_flat(img_paths)

In [500]:
input_data.shape

torch.Size([30, 409600])

# Second Linear Prototype with Flattened Tensor

In [510]:
class linear_prototype2(nn.Module):
    def __init__(self, img_h, img_w):
        super().__init__()
        #define sizes here
        self.h = img_h
        self.w = img_w
        self.longshape = img_h*img_w
        self.linear1 = nn.Linear(self.longshape, 320)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(320, 2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        # preprocess the input image
        x = self.preprocess_image_flat(x)
        #============ Layer1==============#
        x = self.linear1(x)
        x = self.relu1(x)
        #============Layer2==============#
        x = self.linear2(x)
        x = self.softmax(x)
        return x
    
    ## update this to sequential?
    def preprocess_image_flat(self, path):
        resizer = TF.Resize((self.h, self.w)) #define resizer per new_h and new_w
        im = tv.io.read_image(path).type(torch.float) #read image as pytorch float tensor
        im = resizer(im) #resize image
        normalizer = TF.Normalize(im.mean(), im.std()) #initialize normalizer
        im = normalizer(im) # return normalized pytorch float tensor
        im = torch.flatten(im)
        return im.clone().detach().requires_grad_(True)

In [512]:
class linear_prototype3(nn.Module):
    def __init__(self, img_h, img_w):
        super().__init__()
        #define sizes here
        self.h = img_h
        self.w = img_w
        self.longshape = img_h*img_w
        self.linear1 = nn.Linear(self.longshape, 320)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(320, 2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        # preprocess the input image
        #============ Layer1==============#
        x = self.linear1(x)
        x = self.relu1(x)
        #============Layer2==============#
        x = self.linear2(x)
        x = self.softmax(x)
        return x

## Optimizer

In [513]:
model = linear_prototype3(640, 640)

In [518]:
y = model(input_data)

In [519]:
y.shape

torch.Size([30, 2])

In [574]:
criterion = torch.nn.BCELoss(reduction="sum")

In [571]:
def recall_loss(y_true, y_pred):
    """
    y_true shape = observation, 2 (negative, positve)
    y_pred shape = observation, 2 (negative, positive) but in posibilities
    
    """
    
    
    
    answers = list(zip(y_true, y_pred))
    true_positives = sum(1 if (true[] == 1 and pred == 1) else 0 for true, pred in answers)
    false_negatives = sum(1 if (true == 1 and pred == 0) else 0 for true, pred in answers)
    return torch.tensor(1 - (true_positives/(true_positives + false_negatives)))

In [584]:
def run_optim(n_seq, learning_rate, img_paths):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for t in range(n_seq):
        print("training epoch", t)
        y_pred = model(img_paths)
        loss = criterion(y_pred, y_true) #calculate loss

        plt.scatter(t, loss.detach())
        optimizer.zero_grad() #reset gradient)
        
        # gradient back step
        loss.backward()
    
        optimizer.step()
        
        # update parameters per learning rate (go down the gradient)
        with torch.no_grad(): #sequential
            for param in model.parameters():
                param += learning_rate * param.grad

In [585]:
run_optim(10, 0.1, input_data)

training epoch 0


AssertionError: 

***


## First Linear Prototype

The issue with this model is that the output is `[1, 640, 2]` when the output should be a shape of `[2,]`. 

A flattening maybe required

In [108]:
class linear_prototype(nn.Module):
    def __init__(self, img_h, img_w):
        super().__init__()
        #define sizes here
        self.h = img_h
        self.w = img_w
        self.linear1 = nn.Linear(img_w, 320)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(320, 2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        # preprocess the input image
        x = self.preprocess_image(x)
        #============ Layer1==============#
        x = self.linear1(x)
        x = self.relu1(x)
        #============Layer2==============#
        x = self.linear2(x)
        
        return self.softmax(x)
    
    ## update this to sequential?
    def preprocess_image(self, path):
        resizer = TF.Resize((self.h, self.w)) #define resizer per new_h and new_w
        im = tv.io.read_image(path).type(torch.float) #read image as pytorch float tensor
        im = resizer(im) #resize image
        normalizer = TF.Normalize(im.mean(), im.std()) #initialize normalizer
        return normalizer(im) # return normalized pytorch float tensor

In [109]:
proto_model = linear_prototype(640, 640)

In [110]:
proto_model

linear_prototype(
  (linear1): Linear(in_features=640, out_features=320, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=320, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [112]:
y_pred = proto_model(test_image_path)

In [113]:
y_pred.shape

torch.Size([1, 640, 2])

In [472]:
def run_optim2(n_seq, learning_rate, img_path_list):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for t in range(n_seq):
        y_pred = [model(path) for path in img_path_list]
        y_pred = torch.tensor([1 if pred[0] > pred[1] else 0 for pred in y_pred]).float()
        y_pred = y_pred.clone().detach().requires_grad_(True)
        
        loss = criterion(y_pred, y_true) #calculate loss

        plt.scatter(t, loss.detach())
        optimizer.zero_grad() #reset gradient)
        
        # gradient back step
        loss.backward()
    
        optimizer.step()
        
        # update parameters per learning rate (go down the gradient)
        with torch.no_grad(): #sequential
            for param in model.parameters():
                if param.grad is not None:
                    param += learning_rate * param.grad